## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-11-02-20-18 +0000,bbc,Trump threatens 35% tariffs on Canadian goods,https://www.bbc.com/news/articles/cvg819n954mo
1,2025-07-11-02-08-37 +0000,nypost,Trump announces 35% tariffs on imports from Ca...,https://nypost.com/2025/07/10/us-news/trump-an...
2,2025-07-11-01-50-02 +0000,bbc,Judge blocks Trump's birthright order after Su...,https://www.bbc.com/news/articles/c0l4jw5glplo
3,2025-07-11-01-45-42 +0000,nyt,Trump Tariff Letters: See the Latest Rates and...,https://www.nytimes.com/2025/07/07/business/tr...
4,2025-07-11-00-46-14 +0000,nypost,Largest US teachers union mocked for misspelli...,https://nypost.com/2025/07/10/us-news/largest-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2301/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,25
126,brazil,5
2,tariffs,5
18,tariff,3
215,he,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
49,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,48
24,2025-07-10-18-53-57 +0000,nypost,Trump reflects on assassination attempt one ye...,https://nypost.com/2025/07/10/us-news/trump-re...,46
38,2025-07-10-15-01-21 +0000,nypost,Marco Rubio slams Russia over Ukrainian war de...,https://nypost.com/2025/07/10/us-news/marco-ru...,46
23,2025-07-10-19-21-22 +0000,nypost,Senate clears path to confirm first judge of T...,https://nypost.com/2025/07/10/us-news/senate-c...,46
28,2025-07-10-17-43-26 +0000,nypost,Pictured: Trump’s Secret Service agent who was...,https://nypost.com/2025/07/10/us-news/secret-s...,45


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
49,48,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...
66,27,2025-07-10-04-39-28 +0000,nypost,Ex-CIA Director John Brennan says he’s had ‘no...,https://nypost.com/2025/07/10/us-news/ex-cia-d...
38,21,2025-07-10-15-01-21 +0000,nypost,Marco Rubio slams Russia over Ukrainian war de...,https://nypost.com/2025/07/10/us-news/marco-ru...
23,21,2025-07-10-19-21-22 +0000,nypost,Senate clears path to confirm first judge of T...,https://nypost.com/2025/07/10/us-news/senate-c...
6,21,2025-07-10-23-49-13 +0000,nypost,Hakeem Jeffries’ team blasts Zohran Mamdani’s ...,https://nypost.com/2025/07/10/us-news/hakeem-j...
28,20,2025-07-10-17-43-26 +0000,nypost,Pictured: Trump’s Secret Service agent who was...,https://nypost.com/2025/07/10/us-news/secret-s...
14,16,2025-07-10-21-00-31 +0000,nypost,"Texas AG Ken Paxton’s wife, state Sen. Angela,...",https://nypost.com/2025/07/10/us-news/texas-ag...
25,14,2025-07-10-18-25-16 +0000,bbc,When to book and where to stay: Six ways to sa...,https://www.bbc.com/news/articles/cq6mvn699v9o
35,14,2025-07-10-15-38-27 +0000,nypost,New Hampshire judge blocks Trump’s birthright ...,https://nypost.com/2025/07/10/us-news/federal-...
4,13,2025-07-11-00-46-14 +0000,nypost,Largest US teachers union mocked for misspelli...,https://nypost.com/2025/07/10/us-news/largest-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
